# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 19 2022 12:45PM,244572,21,624851,"NBTY Global, Inc.",Released
1,Aug 19 2022 12:43PM,244571,19,ADV80005080,"AdvaGen Pharma, Ltd",Released
2,Aug 19 2022 12:41PM,244570,10,0085942524,ISDIN Corporation,Released
3,Aug 19 2022 12:41PM,244570,10,0085942530,ISDIN Corporation,Released
4,Aug 19 2022 12:41PM,244570,10,0085942529,ISDIN Corporation,Released
5,Aug 19 2022 12:41PM,244570,10,0085942535,ISDIN Corporation,Released
6,Aug 19 2022 12:41PM,244570,10,0085942534,ISDIN Corporation,Released
7,Aug 19 2022 12:41PM,244570,10,0085942540,ISDIN Corporation,Released
8,Aug 19 2022 12:41PM,244570,10,0085942541,ISDIN Corporation,Released
9,Aug 19 2022 12:41PM,244570,10,0085942548,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
14,244567,Released,4
15,244568,Released,1
16,244570,Released,12
17,244571,Released,1
18,244572,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
244567,NaN,4.0
244568,NaN,1.0
244570,NaN,12.0
244571,NaN,1.0
244572,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
244517,0.0,2.0
244530,0.0,7.0
244544,0.0,5.0
244546,0.0,1.0
244551,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
244517,0,2
244530,0,7
244544,0,5
244546,0,1
244551,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,244517,0,2
1,244530,0,7
2,244544,0,5
3,244546,0,1
4,244551,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,244517,,2
1,244530,,7
2,244544,,5
3,244546,,1
4,244551,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 19 2022 12:45PM,244572,21,"NBTY Global, Inc."
1,Aug 19 2022 12:43PM,244571,19,"AdvaGen Pharma, Ltd"
2,Aug 19 2022 12:41PM,244570,10,ISDIN Corporation
14,Aug 19 2022 12:18PM,244567,10,"Methapharm, Inc."
18,Aug 19 2022 12:18PM,244568,10,Emerginnova
19,Aug 19 2022 12:16PM,244565,10,"Methapharm, Inc."
38,Aug 19 2022 12:14PM,244563,16,Sartorius Stedim North America
39,Aug 19 2022 12:11PM,244562,16,Sartorius Stedim North America
40,Aug 19 2022 11:56AM,244557,10,Eywa Pharma Inc.
42,Aug 19 2022 11:43AM,244561,10,MedStone Pharma LLC


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Aug 19 2022 12:45PM,244572,21,"NBTY Global, Inc.",,1
1,Aug 19 2022 12:43PM,244571,19,"AdvaGen Pharma, Ltd",,1
2,Aug 19 2022 12:41PM,244570,10,ISDIN Corporation,,12
3,Aug 19 2022 12:18PM,244567,10,"Methapharm, Inc.",,4
4,Aug 19 2022 12:18PM,244568,10,Emerginnova,,1
5,Aug 19 2022 12:16PM,244565,10,"Methapharm, Inc.",,19
6,Aug 19 2022 12:14PM,244563,16,Sartorius Stedim North America,,1
7,Aug 19 2022 12:11PM,244562,16,Sartorius Stedim North America,,1
8,Aug 19 2022 11:56AM,244557,10,Eywa Pharma Inc.,2,
9,Aug 19 2022 11:43AM,244561,10,MedStone Pharma LLC,,8


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 19 2022 12:45PM,244572,21,"NBTY Global, Inc.",1,
1,Aug 19 2022 12:43PM,244571,19,"AdvaGen Pharma, Ltd",1,
2,Aug 19 2022 12:41PM,244570,10,ISDIN Corporation,12,
3,Aug 19 2022 12:18PM,244567,10,"Methapharm, Inc.",4,
4,Aug 19 2022 12:18PM,244568,10,Emerginnova,1,
5,Aug 19 2022 12:16PM,244565,10,"Methapharm, Inc.",19,
6,Aug 19 2022 12:14PM,244563,16,Sartorius Stedim North America,1,
7,Aug 19 2022 12:11PM,244562,16,Sartorius Stedim North America,1,
8,Aug 19 2022 11:56AM,244557,10,Eywa Pharma Inc.,,2
9,Aug 19 2022 11:43AM,244561,10,MedStone Pharma LLC,8,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 19 2022 12:45PM,244572,21,"NBTY Global, Inc.",1,
1,Aug 19 2022 12:43PM,244571,19,"AdvaGen Pharma, Ltd",1,
2,Aug 19 2022 12:41PM,244570,10,ISDIN Corporation,12,
3,Aug 19 2022 12:18PM,244567,10,"Methapharm, Inc.",4,
4,Aug 19 2022 12:18PM,244568,10,Emerginnova,1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Aug 19 2022 12:45PM,244572,21,"NBTY Global, Inc.",1.0,NaN
1,Aug 19 2022 12:43PM,244571,19,"AdvaGen Pharma, Ltd",1.0,NaN
2,Aug 19 2022 12:41PM,244570,10,ISDIN Corporation,12.0,NaN
3,Aug 19 2022 12:18PM,244567,10,"Methapharm, Inc.",4.0,NaN
4,Aug 19 2022 12:18PM,244568,10,Emerginnova,1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 19 2022 12:45PM,244572,21,"NBTY Global, Inc.",1.0,0.0
1,Aug 19 2022 12:43PM,244571,19,"AdvaGen Pharma, Ltd",1.0,0.0
2,Aug 19 2022 12:41PM,244570,10,ISDIN Corporation,12.0,0.0
3,Aug 19 2022 12:18PM,244567,10,"Methapharm, Inc.",4.0,0.0
4,Aug 19 2022 12:18PM,244568,10,Emerginnova,1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,1956473,52.0,2.0
16,1222737,10.0,0.0
17,244560,1.0,0.0
18,244559,0.0,1.0
19,244571,1.0,0.0
21,733676,3.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,1956473,52.0,2.0
1,16,1222737,10.0,0.0
2,17,244560,1.0,0.0
3,18,244559,0.0,1.0
4,19,244571,1.0,0.0
5,21,733676,3.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,52.0,2.0
1,16,10.0,0.0
2,17,1.0,0.0
3,18,0.0,1.0
4,19,1.0,0.0
5,21,3.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,52.0
1,16,Released,10.0
2,17,Released,1.0
3,18,Released,0.0
4,19,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,16,17,18,19,21
Status,,,,,,
Executing,2.0,0.0,0.0,1.0,0.0,0.0
Released,52.0,10.0,1.0,0.0,1.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,16,17,18,19,21
0,Executing,2.0,0.0,0.0,1.0,0.0,0.0
1,Released,52.0,10.0,1.0,0.0,1.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,16,17,18,19,21
0,Executing,2.0,0.0,0.0,1.0,0.0,0.0
1,Released,52.0,10.0,1.0,0.0,1.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()